#### **Chargement des Claims dans Bronze**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, when

In [2]:
# Create Spark session
spark = SparkSession.builder \
                    .appName("Healthcare Claims Ingestion") \
                    .getOrCreate()

In [3]:
BUCKET_NAME = "healthcare-bucket-lte"
CLAIMS_BUCKET_PATH = f"gs://{BUCKET_NAME}/landing/claims/*.csv"
BQ_TABLE = "gcp-demo-lte.bronze_dataset.claims"
TEMP_GCS_BUCKET = f"{BUCKET_NAME}/temp/"


In [4]:
# read from claims source
claims_df = spark.read.csv(CLAIMS_BUCKET_PATH, header=True)

# adding hospital source for future reference
claims_df = (claims_df
                .withColumn("datasource", 
                              when(input_file_name().contains("hospital2"), "hosb")
                             .when(input_file_name().contains("hospital1"), "hosa").otherwise("None")))

# dropping dupplicates if any
claims_df = claims_df.dropDuplicates()



In [5]:
# write to bigquery
(claims_df.write
            .format("bigquery")
            .option("table", BQ_TABLE)
            .option("temporaryGcsBucket", TEMP_GCS_BUCKET)
            .mode("overwrite")
            .option("overwriteSchema", True)
            .save())